In [1]:
# dependencies go here
import pandas as pd
from sqlalchemy import create_engine
from config import pw
import os

## set up database

We set up the `minneapolis_housing` database and related tables in Postgresql:

* `neighborhood`
* `home_value`
* `crime`

Code for setting up the tables can be found in the queries.sql file accompanying this notebook.

**Note: Refactor this to work with SQLAlchemy, if time permits**

## load data
### Neighborhood datasets

In [2]:
# Stacy start

In [3]:
# Load Minneapolis city dataset for neighborhoods
n_path = os.path.join('.', 'data', 'Minneapolis_Neighborhoods.csv')
neighborhood = pd.read_csv(n_path)
neighborhood_df = neighborhood[['FID', 'BDNAME']].copy()
neighborhood_df.rename(columns={
    'FID': 'id',
    'BDNAME': 'neighborhood'
}, inplace=True)
neighborhood_df.set_index('id', inplace=True)

'''
NOTES:
Noticed there wasn't a codebook that explained what the cryptic column names mean, so used FID based on the fact
that it is a unique ID.
'''

"\nNOTES:\nNoticed there wasn't a codebook that explained what the cryptic column names mean, so used FID based on the fact\nthat it is a unique ID.\n\n"

In [26]:
# Load MNCompass Minneapolis neighborhood dataset 
c_path = os.path.join('.', 'data', 'MSP Neighborhoods_2013-2017.csv')
compass = pd.read_csv(c_path)
compass_df = compass[['City', 'geography', 'Total population - Total population - count']]
compass_df = compass_df.dropna(axis='rows', how='any')
compass_df = compass_df.drop(index=0)
minneapolis_df = compass_df[compass_df['City'] == 'Minneapolis']
minneapolis_df.rename(columns={
    'Total population - Total population - count': 'population',
    'geography': 'neighborhood'
})
# next merge the dfs together into one neighborhood df

,City,neighborhood,population
1,Minneapolis,Corcoran,4292.073566
2,Minneapolis,Kenny,3611.528805
3,Minneapolis,Cooper,3691.666001
4,Minneapolis,East Harriet,3673.414496
5,Minneapolis,Cedar-Isles-Dean,2949.773706
6,Minneapolis,Lind-Bohanon,5322
7,Minneapolis,Holland,4954.809422
8,Minneapolis,Wenonah,4482.226648
9,Minneapolis,Folwell,5768.859611
10,Minneapolis,Tangletown,4493.280841


Columns to keep:
* Total population - Total population - count
* geography
* City

**Note: Refactor this to include other information, like `% population over 65`, which could be useful in forecasting up and coming neighhorhoods** 

In [ ]:
# Stacy end

In [ ]:
# Jenna start

In [ ]:
# Jenna end

In [ ]:
# Katrina start

In [ ]:
# Katrina end

## transforming data

In [ ]:
# Jenna start

In [ ]:
# Jenna end

In [ ]:
# Katrina start

In [ ]:
# Katrina end

## adding to postgres

In [ ]:
# Stacy start

In [ ]:
# Stacy end